In [1]:
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from time import sleep

In [2]:
def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    if canfetch(url) == False:
        print('[Error] ' + url)
#     else: # 실제 수집할 때, 제약사항이 많으므로 여기선 잠시 해제
    try:
        resp = request(method, url,
               params=params if method=='GET' else {},
               data=params if method=='POST' else {},
               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(1) # => random
            resp = download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
    return resp

In [3]:
# F12로 구조 살펴보기
url = 'https://www.google.com/search'
params = {
    'q':'',
    'oq':'',
    'aqs':'chrome..69i57j69i59j69i65l3j69i61j69i60j69i61.1205j0j7',
    'sourceid':'chrome',
    'ie':'UTF-8'
}
params['q'] = params['oq'] = '파이썬'
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.text, 'html.parser')
for _ in dom.find_all('h3', {'class':'LC20lb'}):
    print(_.text.strip(), _.find_parents('a')[0]['href']) 

[Error] https://www.google.com/search
파이썬 - 위키백과, 우리 모두의 백과사전 https://ko.wikipedia.org/wiki/%ED%8C%8C%EC%9D%B4%EC%8D%AC
파이썬 자습서 — Python 3.8.4 문서 https://docs.python.org/ko/3/tutorial/index.html
Python - 나무위키 https://namu.wiki/w/Python
1 파이썬 시작하기 - 왕초보를 위한 Python - WikiDocs https://wikidocs.net/43
Python 코딩의 기초 - 예제로 배우는 파이썬 프로그래밍 http://pythonstudy.xyz/python/article/6-Python-%EC%BD%94%EB%94%A9%EC%9D%98-%EA%B8%B0%EC%B4%88
파이썬의 인기는 언제까지 갈까? - Byline Network https://byline.network/2020/03/17-90/
파이썬 코딩 도장: 1.3 파이썬 https://dojang.io/mod/page/view.php?id=2153
파이썬 무료 강의 (기본편) - 6시간 뒤면 나도 개발자 - 인프런 https://www.inflearn.com/course/%EB%82%98%EB%8F%84%EC%BD%94%EB%94%A9-%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EA%B8%B0%EB%B3%B8


In [4]:
url = 'https://search.naver.com/search.naver'
params = {
    'sm':'top_hty',
    'fbm':0,
    'ie':'utf8',
    'query':''
}
params['query'] = '파이썬'
resp = download(url, params, headers, 'GET')
resp.text

[Error] https://search.naver.com/search.naver


'<!doctype html> <html lang="ko"> <head> <meta charset="utf-8"> <meta name="referrer" content="always">  <meta name="format-detection" content="telephone=no,address=no,email=no"> <meta name="viewport" content="width=device-width,initial-scale=1.0,maximum-scale=2.0"> <meta property="og:title" content="파 이 썬 : 네이버 통합검색"/> <meta property="og:image" content="https://ssl.pstatic.net/sstatic/search/common/og_v3.png"> <meta property="og:description" content="\'파 이 썬\'의 네이버 통합검색 결과입니다."> <meta name="description" lang="ko" content="\'파 이 썬\'의 네이버 통합검색 결과입니다."> <title>파 이 썬 : 네이버 통합검색</title> <link rel="shortcut icon" href="https://ssl.pstatic.net/sstatic/search/favicon/favicon_191118_pc.ico">  <link rel="search" type="application/opensearchdescription+xml" href="https://ssl.pstatic.net/sstatic/search/opensearch-description.https.xml" title="Naver" /><link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/sstatic/search/pc/css/search1_200716.css"> <link rel="stylesheet" type="text/c

In [9]:
url = 'https://search.naver.com/search.naver'
params = {
    'sm':'top_hty',
    'fbm':0,
    'ie':'utf8',
    'query':''
}
params['query'] = '파이썬'
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.content,'html.parser')

[Error] https://search.naver.com/search.naver


In [ ]:
# 파이썬 크롤링 결과를 보면 class ul에 class:type로 나눠져있다. 
# dt: title

In [10]:
for _ in dom.find_all('ul',{'class':'type01'}):
    for a in [dt.find('a') for dt in _.find_all('dt')]:
        print(a['href'], a.text)

https://www.python.org/ Welcome to Python.org
https://namu.wiki/w/Python Python - 나무위키
https://www.python.org/downloads/ Looking for a specific release? - Download Python | Python.org
https://blog.naver.com/sweetk2ss?Redirect=Log&logNo=221954566322 파이썬 설치 다운로드부터 해야겠죠?
http://blog.wishket.com/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%9D%B8%EA%B8%B0%EA%B0%80-%EB%96%A8%EC%96%B4%EC%A7%80%EB%A9%B4-%EC%96%B4%EB%96%A4-%EA%B0%9C%EB%B0%9C-%EC%96%B8%EC%96%B4%EA%B0%80-%EB%9C%B0%EA%B9%8C/ '파이썬'의 인기가 떨어지면 어떤 개발 언어가 뜨게 될까....
https://blog.naver.com/thkimnguy?Redirect=Log&logNo=222016218489 점프 투 파이썬으로 코딩 공부 시작하기
https://blog.naver.com/icbanq?Redirect=Log&logNo=222033791949 [파이썬] 파이썬의 데이터 구조를 알아보자(1편)
https://blog.naver.com/fastcampus?Redirect=Log&logNo=221924690201 직장인이라면 꼭 알아야 할 기술, 업무자동화 (엑셀/VBA/파이썬)
https://m.post.naver.com/viewer/postView.nhn?volumeNo=28458984&memberNo=34865381&vType=VERTICAL [출간전연재] 1. 프로그래밍 언어에 대한 이해-파이썬이란?
https://m.post.naver.com/viewer/postView.nhn?volumeNo=28718043&memberNo=21815&vTy

In [26]:
url = 'https://search.daum.net/search'
params = {
    'w':'tot',
    'DA':'YZR',
    't__nil_searchbox':'btn',
    'sug':'',
    'sugo':'',
    'sq':'',
    'o':'',
    'q':''
}
params['q'] = '파 이 썬'
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.content,'html.parser')

[Error] https://search.daum.net/search


In [27]:
for _ in dom.find_all('div',{'class':'wrap_tit'}):
    print(_.find('a')['href'], _.find('a').text.strip()) # url과 제목

https://keyword.ad.daum.net/short/clk?q=CjfFlbkKj4NhKc1zigwmt9BaUyIJHxIYPtGFVNE7_fslcp1iaDAUb2VTPvcJBvCZD2f4iGf9GZZahUFNH86oHlY4sQQJZTug-s9a3J2PGNGMScuqE_zHDVUPzub144Xx4qzh3R3P3g7MmOvyWfgW4cde5uThsw.ONCE.lIADSMlF_zRXFE-LEwC8sy9Ot_p.mHOrPcLKwkMIWhLE.2o5EtLvTdtKlOUyjAQ6nkZLbYQZJg00&s=http%3A%2F%2Fwww.itbank21.com%2F%3FDMKW%3D%25ED%258C%258C%25EC%259D%25B4%25EC%258D%25AC%26DMSKW%3D%25ED%258C%258C%25EC%259D%25B4%25EC%258D%25AC%26DMCOL%3DPM KG아이티뱅크 코딩학원 김서균
https://keyword.ad.daum.net/short/clk?q=CjvexXMuANR75UvZ-cpkj-9N5YMNmdylgtyCteq8EIKI7uupFt7Y.MDrDp-IYt-Xx7RSZS1jg4.as8kgf3SUEas7W2wE4FqGNFLdPz6E1dQl1NYWkfTrBjO3we1wxzXXmmoo91UIDDqREk4ntbJ597zINYaTs_SVWrMN.ge6Mv4uNEg_6z8YoOPvHM_rPtvXOFoUI4vBuqyJG6Kk2R2z8VM7MIjyC29FrdZLvf.TjC57fuFg6C7fFtIwdMB1xaaNDjaIN.qQL9w9Dwa7BblLT5nRbwMA_f2r4i9-NQcOdKEKRIJuXzd-rpWiCWPBJn6frb7shhbYSGxS17Aw-MQrLOaW&s=http%3A%2F%2Fwww.tjoeun.co.kr%2Ffront%2Fetc%2Fv2020s%2F%3Futm_source%3Ddaum%26utm_medium%3Dcpc%26utm_campaign%3Dpremiumlink%26utm_content%3Dseason%26utm_ter

In [34]:
resp = download('http://pythonscraping.com/pages/page3.html')
dom = BeautifulSoup(resp.text, 'lxml')

In [ ]:
# tag, #id, .class, .class.class.class
# ul.class, ul.class.class.class
# tag, tag, tag => CS

In [35]:
dom.select_one('div#footer') == dom.select_one('#footer')

True

In [36]:
dom.select_one('#footer').text.strip()

'© Totally Normal Gifts, Inc. \n+234 (617) 863-0736'

In [ ]:
Selector
tag1, tag2
tag1 tag2 => 자손 (find_all(recursive = True))
tag1 > tag2 => 자식 (find_all)recursive = False))
tag1 + tag2 => 형제(다음 노드) => tag2

In [38]:
dom.select_one('#footer').find_parent().name

'div'

In [39]:
len(dom.select('#wrapper > div')), len(dom.select('#wrapper > *'))

(2, 5)

In [40]:
[_.name for _ in dom.select('#wrapper > *')]

['img', 'h1', 'div', 'table', 'div']

In [43]:
dom.select_one('h1 + div').name

'div'

In [44]:
dom.select_one('h1 + div').find_previous_sibling().name

'h1'

In [45]:
dom.select_one('body > div > h1 + div').name

'div'

In [47]:
# td들 중 3번째
[_.text.strip() for _ in dom.select('.gift > td:nth-of-type(3)')]

['$15.00', '$10,000.52', '$10,005.00', '$0.50', '$1.50']

In [49]:
# td 아래 이미지들
[_['src'].strip() for _ in dom.select('.gift > td > img')]

['../img/gifts/img1.jpg',
 '../img/gifts/img2.jpg',
 '../img/gifts/img3.jpg',
 '../img/gifts/img4.jpg',
 '../img/gifts/img6.jpg']

In [51]:
# 앞 구글 예시를 이렇게도 코딩할 수 있다.
url = 'https://www.google.com/search'
params = {
    'q':'',
    'oq':'',
    'aqs':'chrome..69i57j69i59j69i65l3j69i61j69i60j69i61.1205j0j7',
    'sourceid':'chrome',
    'ie':'UTF-8'
}
params['q'] = params['oq'] = '파이썬'
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.text, 'html.parser')
# for _ in dom.find_all('h3', {'class':'LC20lb'}):
#     print(_.text.strip(), _.find_parents('a')[0]['href']) 
for _ in dom.select('div.r > a'):
    print(['href'], _.text.strip())

[Error] https://www.google.com/search
['href'] 파이썬 - 위키백과, 우리 모두의 백과사전ko.wikipedia.org › wiki › 파이썬
['href'] 파이썬 자습서 — Python 3.8.4 문서docs.python.org › tutorial
['href'] Python - 나무위키namu.wiki › Python
['href'] 1 파이썬 시작하기 - 왕초보를 위한 Python - WikiDocswikidocs.net › ...
['href'] Python 코딩의 기초 - 예제로 배우는 파이썬 프로그래밍pythonstudy.xyz › python › article › 6-Python-코딩의-...
['href'] 파이썬 코딩 도장: 1.3 파이썬dojang.io › 강좌 › 프로그래밍 › 파이썬 코딩 도장
['href'] 파이썬의 인기는 언제까지 갈까? - Byline Networkbyline.network › 2020/03
['href'] 파이썬 무료 강의 (기본편) - 6시간 뒤면 나도 개발자 - 인프런www.inflearn.com › course › 나도코딩-파이썬-기본


In [52]:
# Crawling

In [54]:
url = 'http://example.webscraping.com/places/default/index'
resp = download(url)
dom = BeautifulSoup(resp.content, 'html.parser')

In [55]:
for _ in dom.select('a'):
    print(_['href']) # 모든 a tag

#
/places/default/user/register?_next=/places/default/index
/places/default/user/login?_next=/places/default/index
/places/default/index
/places/default/search
/places/default/view/Afghanistan-1
/places/default/view/Aland-Islands-2
/places/default/view/Albania-3
/places/default/view/Algeria-4
/places/default/view/American-Samoa-5
/places/default/view/Andorra-6
/places/default/view/Angola-7
/places/default/view/Anguilla-8
/places/default/view/Antarctica-9
/places/default/view/Antigua-and-Barbuda-10
/places/default/index/1


In [61]:
# 위 사이트 중 한 군데 클릭
url = 'http://example.webscraping.com/places/default/view/Afghanistan-1'
resp = download(url)
dom = BeautifulSoup(resp.content, 'html.parser')
for _ in dom.select('a'):
    print(_['href'])

#
/places/default/user/register?_next=/places/default/view/Afghanistan-1
/places/default/user/login?_next=/places/default/view/Afghanistan-1
/places/default/index
/places/default/search
/places/default/continent/AS
/places/default/iso/TM
/places/default/iso/CN
/places/default/iso/IR
/places/default/iso/TJ
/places/default/iso/PK
/places/default/iso/UZ
/places/default/edit/Afghanistan-1


In [58]:
# urls.pop() 꺼낼 것, urls. append() 추가할 것

In [74]:
urls = list()
seen = list() # 방문했는지
urls.append(url)

while urls: # Queue -> fFIrst in first out -> pop(0), urls이 존재하는 한
    seed = urls.pop(0) # starting url
    seen.append(seed) # 재방문 회피
    dom = BeautifulSoup(download(seed).text, 'html.parser') # HTTP
#     for _ in dom.select('a'): # extract hyperlinks
#         if _.has_attr('href'): # 주소 있는 지
#             if _['href'].startswith('/'): # filter
#                 newUrls = urljoin(seed, _['href']) # Normalization
#                 # query 부분 (GET방식에서 ? 이후에 나오는 파라미터 생략하는 코드)
#                 if newUrls not in seen and newUrls not in urls: # unseen인 애들
#                     urls.append(newUrls) # 계속 돈다. 새롭게 돈 애들 추가
# #                    print(newUrls)
    for _ in [_['href'] for _ in dom.select('a')
             if _.has_attr('href') and _['href'].startswith('/')]:
            newUrls = urljoin(seed, urlparse(_)[2])
            if newUrls not in seen and newUrls not in urls:
                urls.append(newUrls)
    print(len(urls), len(seen)) # 계속 업데이트 됨

12 1
11 2
10 3
19 4
18 5
68 6
70 7
80 8
84 9
84 10
84 11
84 12
83 13
85 14
91 15
99 16
100 17
102 18
106 19
107 20
108 21
108 22
115 23
116 24
116 25
116 26
116 27
116 28
116 29
117 30
118 31
117 32
117 33
117 34
117 35
117 36
118 37
120 38
119 39
123 40
126 41
126 42


KeyboardInterrupt: 

In [75]:
# 남아있는 애들
urls

['http://example.webscraping.com/places/default/view/Kazakhstan-115',
 'http://example.webscraping.com/places/default/view/Kuwait-119',
 'http://example.webscraping.com/places/default/view/Kyrgyzstan-120',
 'http://example.webscraping.com/places/default/view/Laos-121',
 'http://example.webscraping.com/places/default/view/Lebanon-123',
 'http://example.webscraping.com/places/default/view/Macao-130',
 'http://example.webscraping.com/places/default/view/Malaysia-134',
 'http://example.webscraping.com/places/default/view/Maldives-135',
 'http://example.webscraping.com/places/default/view/Mongolia-147',
 'http://example.webscraping.com/places/default/view/Myanmar-152',
 'http://example.webscraping.com/places/default/view/Nepal-155',
 'http://example.webscraping.com/places/default/view/North-Korea-165',
 'http://example.webscraping.com/places/default/view/Oman-168',
 'http://example.webscraping.com/places/default/view/Pakistan-169',
 'http://example.webscraping.com/places/default/view/Palest

In [76]:
# 본 애들
seen

['http://example.webscraping.com/places/default/view/Afghanistan-1',
 'http://example.webscraping.com/places/default/user/register',
 'http://example.webscraping.com/places/default/user/login',
 'http://example.webscraping.com/places/default/index',
 'http://example.webscraping.com/places/default/search',
 'http://example.webscraping.com/places/default/continent/AS',
 'http://example.webscraping.com/places/default/iso/TM',
 'http://example.webscraping.com/places/default/iso/CN',
 'http://example.webscraping.com/places/default/iso/IR',
 'http://example.webscraping.com/places/default/iso/TJ',
 'http://example.webscraping.com/places/default/iso/PK',
 'http://example.webscraping.com/places/default/iso/UZ',
 'http://example.webscraping.com/places/default/edit/Afghanistan-1',
 'http://example.webscraping.com/places/default/view/Aland-Islands-2',
 'http://example.webscraping.com/places/default/view/Albania-3',
 'http://example.webscraping.com/places/default/view/Algeria-4',
 'http://example.w

In [77]:
# 이런 문법과 비슷한 원리
while len(list()) > 0:
    print('Hello') 

# 정규식 Regular Expression
-  A special text string for describing a search pattern
-  for searching, replacing, and parsing text with complex patterns of characters


In [78]:
# good for preprocessing
# processes large amounts of text over and over again
# Extremely fast
# Usually this pattern is then used by string searching algorithms
# For find or find and replace operations on strings, or for input validations

# Limitations: learning curve

In [ ]:
예를들어
http => ht{2}p
https => https? => 1. s가 없다면 http, s가 있다면 https
hellllllllllo => hel{1,}o
다.                      그래서 => _{+} => _ # white space 1개 이상을 1개로
다. 그래서 # 이렇게 만들어 줌
h[t ]{1,}p => http, htttp, ...
^x => sql문에서 x% 처럼 x로 시작하는 것
[^x] => not x
x$ => sql문에서 %x 처럼 x로 끝나는 것
* => 0개 이상
+ => 1개 이상
/b => 공백+문자+공백
/B => 문자 + ? + 문자
/b[A-Za-z] => 

In [79]:
import re

In [80]:
###
data = '''
park 800905-1049118
kim 700905-1059119
'''
###
pattern = re.compile('(\d{6})[-]\d{7}')
pattern.search(data)
data[6:20]

'800905-1049118'

In [85]:
# replace
pattern = re.compile('(\d{6})[-]\d{7}')
pattern.search(data)
print(pattern.sub('\g<1>-*******',data))


park 800905-*******
kim 700905-*******



In [89]:
result = []
for line in data.split("\n"):
    word_result = []
    for word in line.split(" "):
        if len(word) == 14 and word[:6].isdigit() and word[7:].isdigit():
            word = word[:6] + "-" + "*******"
        word_result.append(word)
    result.append("  ".join(word_result))
print("\n".join(result))
# 같은 결과


park  800905-*******
kim  700905-*******



In [84]:
re.match(r'Crow|Servo','ServoCrowHello')

<re.Match object; span=(0, 5), match='Servo'>

In [88]:
re.search(r'Hello$', 'ServoCrowHello') # Hello로 끝남

<re.Match object; span=(9, 14), match='Hello'>

In [83]:
re.match(r'Crow|Servo','HelloServoCrowHello')

In [90]:
re.search(r'\Bclass\b', 'one subclass is')

<re.Match object; span=(7, 12), match='class'>

In [91]:
re.search(r'\Bclass\B','the declassified algorithm')

<re.Match object; span=(6, 11), match='class'>

In [94]:
result = re.search(r'(\w+) (\w+)', 'Issac Newton, physicist')
result.groups(), \
result.group(0), result.group(1), result.group(2)

(('Issac', 'Newton'), 'Issac Newton', 'Issac', 'Newton')

In [95]:
re.sub(r'(\w+) (\w+)', r'\g<2> \g<1>','Issac Newton, physicist')

'Newton Issac, physicist'

In [96]:
data = '010-0000-0000'
re.search(r'\d+[-]\d+[-]\d+',data)

<re.Match object; span=(0, 13), match='010-0000-0000'>

In [97]:
# what if 
data = '010-0000-0000'
data = '010 0000 0000'
data = '010.0000.0000'
data = '01000000000'
re.search(r'\d+[- .]?\d+[- .]?\d+',data)  # ?: 있거나 없거나

<re.Match object; span=(0, 11), match='01000000000'>

In [99]:
data = '+82 2-0001-0002'
data = '+82 10-0001-0002'
data = '+82 010-0001-0002'
data = '82 010-0001-0002'

re.search(r'[+]?\d+[- .]?\d+[- .]?\d+[- .]?\d+',data)

<re.Match object; span=(0, 16), match='82 010-0001-0002'>

In [ ]:
# website example
# dom없이 정규식으로 찾고자 하는 것을 detect!

In [115]:
resp = download('http://pythonscraping.com/pages/page3.html')

In [116]:
re.findall(r'[$]\d+[.]\d+', resp.text)

['$5.00', '$15.00', '$0.50', '$1.50']

In [117]:
'.../img/gifts/img1.jpg'

'.../img/gifts/img1.jpg'

In [118]:
for _ in re.findall(r'src="(.+?img\d[.]jpg)"',resp.text):
    print(urljoin(resp.request.url, _))

http://pythonscraping.com/img/gifts/img1.jpg
http://pythonscraping.com/img/gifts/img2.jpg
http://pythonscraping.com/img/gifts/img3.jpg
http://pythonscraping.com/img/gifts/img4.jpg
http://pythonscraping.com/img/gifts/img6.jpg


In [119]:
'<span class="excitingNote">'

'<span class="excitingNote">'

In [120]:
re.findall(r'<span class="[^"]+">(.+)</span>',resp.text)

['Now with super-colorful bell peppers!',
 '8 entire dolls per set! Octuple the presents!',
 'Also hand-painted by trained monkeys!',
 "Or maybe he's only resting?",
 'Keep your friends guessing!']

In [110]:
dom = BeautifulSoup(resp.text,'html.parser')

In [112]:
dom.find_all(text=re.compile(r'[.]\d+$')) # .(숫자 한개이상)으로 끝나는 구문

['\n$15.00\n', '\n$10,000.52\n', '\n$10,005.00\n', '\n$0.50\n', '\n$1.50\n']

In [113]:
[_.strip() for _ in dom.find_all(text=re.compile(r'[.]\d+$'))]

['$15.00', '$10,000.52', '$10,005.00', '$0.50', '$1.50']

In [ ]:
# email
id@domain # id: 영어로 시작, 특수문자x, -,_,숫자만 허용
company.com|net|org|kr
        .co.kr|or.kr|go.kr.ac.kr

In [124]:
data = '''
    test1@test.com
    test2@test.co.kr
    test3@email.test.com
    test4@email.test.co.kr
    12test@test.com
    한글@test.com
'''

In [125]:
re.findall(r'[A-Za-z][A-Za-z0-9-_.]+@',data) # format 맞지 않은 id 걸러냄

['test1@', 'test2@', 'test3@', 'test4@', 'test@']

In [126]:
re.findall(r'\b\w+@',data) # if 모든 word 허용

['test1@', 'test2@', 'test3@', 'test4@', '12test@', '한글@']

In [129]:
re.findall(r'\b[A-Za-z][A-Za-z0-9-_.]+@(?:[.]?[A-Za-z0-9-_]+)+',data)

['test1@test.com',
 'test2@test.co.kr',
 'test3@email.test.com',
 'test4@email.test.co.kr']

In [130]:
# domain

In [132]:
data = '''
    http://www.domain.com
    http://www.domain.co.kr
    https://www.domain.com
    https://www.domain.co.kr
    https://domain.co.kr/service
    https://www.domain.co.kr/service
    https://www.domain.co.kr/service/
    https://domain.co.kr/service?key=value
    https://www.domain.co.kr/service?key=value
    https://www.domain.co.kr/service?key=value/
    https://www.domain.co.kr/service/1/2?key=value
    https://www.domain.co.kr/service/1/2?key=value/
    ?key=value
    www.domain.com?key-value
    http://old.mail.domain.com?key=value
    http://domain.com#key=value
    http://www.domain.com?key=value
    http://www.domain.com#key=value
'''

In [139]:
# Query String (? 이후) 부분 빼고 URI(L)만 추출해보자
re.findall(r'https?://\w+(?:[./]\w+)+',data) 
# s 있거나 없거나, \w+ word 1개 이상, ()+ 안에 있는 거 1개 이상, 
# ?: . or /포함된 '모든 절' = not make a group by the match inside the parenteses

['http://www.domain.com',
 'http://www.domain.co.kr',
 'https://www.domain.com',
 'https://www.domain.co.kr',
 'https://domain.co.kr/service',
 'https://www.domain.co.kr/service',
 'https://www.domain.co.kr/service',
 'https://domain.co.kr/service',
 'https://www.domain.co.kr/service',
 'https://www.domain.co.kr/service',
 'https://www.domain.co.kr/service/1/2',
 'https://www.domain.co.kr/service/1/2',
 'http://old.mail.domain.com',
 'http://domain.com',
 'http://www.domain.com',
 'http://www.domain.com']

In [150]:
# 위 예시 가져오기
url = 'https://www.google.com/search'
params = {
    'q':'',
    'oq':'',
    'aqs':'chrome..69i57j69i59j69i65l3j69i61j69i60j69i61.1205j0j7',
    'sourceid':'chrome',
    'ie':'UTF-8'
}
params['q'] = params['oq'] = '파이썬'
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.text, 'html.parser')
urls = [{'url':_['href'], 'depth':1}
         for _ in dom.select('div.r > a[href]')]

[Error] https://www.google.com/search


In [141]:
len(urls)

8

In [162]:
seen = list() 

while urls: # Queue -> fFIrst in first out -> pop(0), urls이 존재하는 한
    seed = urls.pop(0) # starting url
    seen.append(seed) # 재방문 회피
    if seed['depth'] <4: # 범위 제한 Focused Crawling
        dom = BeautifulSoup(download(seed['url']).text, 'html.parser')
        for _ in [_['href'] for _ in dom.select('a[href]')
                 if re.match(r'(?:https?:/)?/\w+(?:[./]\w+)+', _['href'])]:
            newUrls = urljoin(seed['url'], urlparse(_)[2])
            if newUrls not in seen and newUrls not in urls:
                urls.append({'url':newUrls, 'depth':seed['depth']+1})
    print(len(urls), len(seen)) 

340 1
346 2
[Error] https://namu.wiki/w/Python
700 3
[Error] https://wikidocs.net/43
700 4
738 5
812 6
842 7
1855 8
2786 9
3131 10
[Error] https://ko.wikipedia.org/w/index.php
3270 11
3409 12
[Error] https://ko.wikipedia.org/w/index.php
3548 13
3647 14
3748 15
[404] https://ko.wikipedia.org/wiki/MicroPython
404
Not Found
{'Date': 'Mon, 20 Jul 2020 07:02:54 GMT', 'Server': 'mw1325.eqiad.wmnet', 'X-Content-Type-Options': 'nosniff', 'P3p': 'CP="See https://ko.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-Language': 'ko', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'X-Request-Id': 'c10fe61b-36cf-47c4-b54d-fb71a3dff30a', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Encoding': 'gzip', 'Age': '396', 'X-Cache': 'cp5008 hit, cp5008 hit/2', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=20-Jul-2020;Path=/;HttpOnly;secure;

KeyboardInterrupt: 

In [ ]:
# 구글(파이썬) -> 8개 검색결과(depth=1) -> 다시 링크타고 이동(depth=2) -> depth=3

In [163]:
temp = '/www.domain.co.kr/service/1/2?key=value/'
re.match(r'https?://\w(?:[./]\w+)+',temp)